In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import StratifiedKFold
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import tensorflow as tf

split_num = 10


# data input 
path = 'trainrat_new.txt'
data = pd.read_csv(path,sep=' ')
print(len(data))

# Data extraction,Win
win_data = data[data.label == "W"]
win_data = win_data.assign(W=1,D=0,L=0) 
win_data = win_data.drop("label",axis=1)
win_data = win_data.sample(n=len(win_data))# random sort

# Data extraction,Draw
draw_data = data[data.label == "D"]
draw_data = draw_data.assign(W=0,D=1,L=0)
draw_data = draw_data.drop("label",axis=1)
draw_data = draw_data.sample(n=len(draw_data))# random sort

# Data extraction,Lose
lose_data = data[data.label == "L"]
lose_data = lose_data.assign(W=0,D=0,L=1) 
lose_data = lose_data.drop("label",axis=1)
lose_data = lose_data.sample(n=len(lose_data))# random sort

# Group data together,use for normal k-fold
all_data = pd.concat([win_data, draw_data, lose_data])
all_data = all_data.sample(n=len(all_data))# random sort
#yAll = xAll[["W","D","L"]]

# Data drop
#xWin = xWin.drop(["W","D","L"],axis=1)
#xDraw = xDraw.drop(["W","D","L"],axis=1)
#xLose = xLose.drop(["W","D","L"],axis=1)
#xAll = xAll.drop(["W","D","L"],axis=1)

# Data separate and making dataset
win_data_separate = []
draw_data_separate = []
lose_data_separate = []
all_data_separate = []
wdl_separate = []
for i in range(split_num):
    win_data_separate.append(win_data[i::split_num])
    draw_data_separate.append(draw_data[i::split_num])
    lose_data_separate.append(lose_data[i::split_num])
    all_data_separate.append(all_data[i::split_num])
    # merge for stratified sampling
    wdl_separate.append(pd.concat([win_data_separate[i],draw_data_separate[i],lose_data_separate[i]]))
    # assign a number to make final input data
    wdl_separate[i] = wdl_separate[i].assign(separate_num=i)
    all_data_separate[i] = all_data_separate[i].assign(separate_num=i)

# integrate everything once
wdl_separate_merge = wdl_separate[0]
all_data_separate_merge = all_data_separate[0]
for i in range(1,split_num):
    wdl_separate_merge = wdl_separate_merge.append(wdl_separate[i])
    all_data_separate_merge = all_data_separate_merge.append(all_data_separate[i])

#print(len(wdl_separate_merge))
#print(len(all_data_separate_merge))
    
    
# make final input data
x_train = []
y_train = []
x_test = []
y_test = []
xAll_train = []
yAll_train = []
xAll_test = []
yAll_test = []
for i in range(split_num):
    x_train.append(wdl_separate_merge[wdl_separate_merge['separate_num'] != i])
    x_test.append(wdl_separate_merge[wdl_separate_merge['separate_num'] == i])
    xAll_train.append(all_data_separate_merge[all_data_separate_merge['separate_num'] != i])
    xAll_test.append(all_data_separate_merge[all_data_separate_merge['separate_num'] == i])
for i in range(split_num):
    # delete separate_num
    x_train[i] = x_train[i].drop(['separate_num'],axis=1)
    x_test[i] = x_test[i].drop(['separate_num'],axis=1)
    xAll_train[i] = xAll_train[i].drop(['separate_num'],axis=1)
    xAll_test[i] = xAll_test[i].drop(['separate_num'],axis=1)
    # random sort
    x_train[i] = x_train[i].sample(n=len(x_train[i]))
    x_test[i] = x_test[i].sample(n=len(x_test[i]))
    xAll_train[i] = xAll_train[i].sample(n=len(xAll_train[i]))
    xAll_test[i] = xAll_test[i].sample(n=len(xAll_test[i]))
    
    # separate x and y
    y_train.append(x_train[i][["W","D","L"]])
    y_test.append(x_test[i][["W","D","L"]])
    yAll_train.append(xAll_train[i][["W","D","L"]])
    yAll_test.append(xAll_test[i][["W","D","L"]])
    x_train[i] = x_train[i].drop(["W","D","L"],axis=1)
    x_test[i] = x_test[i].drop(["W","D","L"],axis=1)
    xAll_train[i] = xAll_train[i].drop(["W","D","L"],axis=1)
    xAll_test[i] = xAll_test[i].drop(["W","D","L"],axis=1)


print(x_train[0].head())
print(y_train[0].head())
print(xAll_train[0].head())
print(yAll_test[0].head())


#print(len(y_train[0].query('W == 1')))
#print(len(y_train[0].query('L == 1')))
#print(len(y_train[0].query('D == 1')))
#print(len(yAll_train[0].query('W == 1')))
#print(len(yAll_train[0].query('L == 1')))
#print(len(yAll_train[0].query('D == 1')))




Using TensorFlow backend.


28186
          HHATT     HHDEF     HAATT     HADEF     AHATT     AHDEF     AAATT  \
15722 -0.095420  0.084677 -0.174840  0.019504  1.090042  0.134123  0.013295   
14291  0.271249  0.267224  0.033173 -0.541828  0.508592  0.645889  0.473082   
16116 -0.465778  0.045621 -0.272313  0.590862 -0.193501  0.093993 -0.016989   
6030   0.392825 -0.004375  0.242558  0.737674  0.280427  0.036327 -0.032873   
6856   2.499735  0.027510  1.302247 -0.601230 -0.403278  0.055617 -0.033696   

          AADEF  
15722  0.472722  
14291  0.589841  
16116  0.265231  
6030   0.216293  
6856   0.649397  
       W  D  L
15722  1  0  0
14291  0  1  0
16116  0  0  1
6030   1  0  0
6856   1  0  0
          HHATT     HHDEF     HAATT     HADEF     AHATT     AHDEF     AAATT  \
25664  0.884264  0.018402  0.111183  0.153576  0.938785 -0.025393  0.486417   
24630  0.675690 -0.121104  0.368988  0.061429  0.619200  0.132433 -0.075396   
13948  0.254323 -0.105522  0.122447 -0.256728  0.442470 -0.204123  0.529970   
11924

In [32]:
# create model
from keras import backend as K

neuron = 10
objective_kind = 3

modelArelu = Sequential()
modelArelu.add(Dense(neuron, activation='relu'))
modelArelu.add(Dense(neuron, activation='relu'))
modelArelu.add(Dense(objective_kind, activation='softmax'))

def RPS(y_true, y_pred):
    sess = tf.Session()
    print(sess.run(y_true))
    cumulative_sum_y_true = 0.
    cumulative_sum_y_pred = 0.
    output = 0.
    holder = tf.placeholder(dtype=tf.float32)
    for i in range(objective_kind - 1):
        print(y_true[i])
        print(y_true)
        y_t = tf.constant(cumulative_sum_y_true,dtype=tf.float32)
        y_p = tf.constant(cumulative_sum_y_pred,dtype=tf.float32)
        add_y_t = tf.add(holder, y_true[i])
        add_y_p = tf.add(holder, y_pred[i])
        with tf.Session() as sess:
            result = sess.run(add_y_t, feed_dict={holder:y_t})
            print(result)
            add1 = sess.run(add_y_t)
            print(add1)
            cumulative_sum_y_true += sess.run(add_y_t)
            cumulative_sum_y_pred += sess.run(add_y_p)
            print(cumulative_sum_y_pred)
            
        output += (cumulative_sum_y_pred - cumulative_sum_y_true) ** 2
    output /= (objective_kind - 1)
    output_tf = tf.constant(output)
    return output_tf


modelArelu.compile(loss=RPS,
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [33]:
cvscores = []
batch_size = 16
num_classes = 10
epochs = 10

for i in range(split_num):
    #print("TRAIN:", x_train, "TEST:", x_test)

    # Fit the model
    modelArelu.fit(x_train[i].values, y_train[i].values,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1
                  )


    results = modelArelu.predict(x_test[i], batch_size=batch_size, verbose=0, steps=None)
    
    print(results)
    print(y_test[i].values)


InvalidArgumentError: You must feed a value for placeholder tensor 'dense_54_target' with dtype float and shape [?,?]
	 [[node dense_54_target (defined at /home/tubotu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517) ]]

Caused by op 'dense_54_target', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/tubotu/.local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/tubotu/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/tubotu/.local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/tubotu/.local/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_f